In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
def compute_loss(y, tx, w, type="mse"):
    """Calculate the loss using mse."""
    N = y.shape[0]
    fw = np.matmul(tx,w)
    if type=="mse":
        return (1/(2*N)*np.matmul(y-fw,(y-fw).T))
    if type=="mae":
        return (1/N*np.sum(np.abs(y-fw)))

def compute_gradient(y, tx, w):
    """Compute the gradient."""
    N=y.size
    e=y-np.matmul(tx,w)
    return(-1/N*np.matmul(tx.T,e))

def compute_stoch_gradient(y, tx, w):
    """Compute a stochastic gradient from just few examples n and their corresponding y_n labels."""
    N=y.size
    e=y-np.matmul(tx,w)
    return(-1/N*np.matmul(tx.T,e))

In [3]:
def least_squares_GD(y, tx, initial_w, max_iters, gamma):
    #linear regression using gradient descent
    w = initial_w
    w = initial_w
    for n_iter in range(max_iters):
        gr=compute_gradient(y,tx,w)
        loss=(compute_loss(y,tx,w))
        w=w-gamma*gr
        #print("Gradient Descent({bi}/{ti}): loss={l}, w0={w0}, w1={w1}".format(
        #      bi=n_iter, ti=max_iters - 1, l=loss, w0=w[0], w1=w[1]))
    return (w, loss)

In [4]:
def batch_iter(y, tx, batch_size, num_batches=1, shuffle=True):
    """
    Generate a minibatch iterator for a dataset.
    Takes as input two iterables (here the output desired values 'y' and the input data 'tx')
    Outputs an iterator which gives mini-batches of `batch_size` matching elements from `y` and `tx`.
    Data can be randomly shuffled to avoid ordering in the original data messing with the randomness of the minibatches.
    Example of use :
    for minibatch_y, minibatch_tx in batch_iter(y, tx, 32):
        <DO-SOMETHING>
    """
    data_size = len(y)

    if shuffle:
        shuffle_indices = np.random.permutation(np.arange(data_size))
        shuffled_y = y[shuffle_indices]
        shuffled_tx = tx[shuffle_indices]
    else:
        shuffled_y = y
        shuffled_tx = tx
    for batch_num in range(num_batches):
        start_index = batch_num * batch_size
        end_index = min((batch_num + 1) * batch_size, data_size)
        if start_index != end_index:
            yield shuffled_y[start_index:end_index], shuffled_tx[start_index:end_index]


In [5]:
def least_squares_SGD(y, tx, initial_w, max_iters, gamma):
    #linear regression using gradient descent
    batch_size = 1
    w = initial_w
    for n_iter in range(max_iters):
        for minibatch_y, minibatch_tx in batch_iter(y, tx, batch_size):
            gr=compute_stoch_gradient(minibatch_y,minibatch_tx,w)
            w=w-gamma*gr
        loss=compute_loss(y,tx, w)
            
        print("Gradient Descent({bi}/{ti}): loss={l}, w0={w0}, w1={w1}".format(
              bi=n_iter, ti=max_iters - 1, l=loss, w0=w[0], w1=w[1]))
    return (w, loss)

In [6]:
# TODO: CHECK WHETHER CONSTANTS ARE OK!!!

In [7]:
def sigmoid(z):
    return 1.0 / (1 + np.exp(-z))

In [8]:
def cross_entropy_loss(y, tx, w):
    N = y.shape[0]
    h = sigmoid(tx @ w)
    return 1/N * ((-y.T @ np.log(h)) + (-(1 - y).T @ np.log(1 - h)))
                  
def cross_entropy_gradient(y, tx, w):
    N = y.shape[0]
    h = sigmoid(tx @ w)
    return 1/N * ((h - y).T @ tx)

In [9]:
def regularized_cross_entropy_loss(y, tx, w, lambda_):
    N = y.shape[0]
    h = sigmoid(tx @ w)
    return 1/N * ((-y.T @ np.log(h)) + (-(1 - y).T @ np.log(1 - h))) + lambda_/(2 * N) * np.sum(w**2)
     
def regularized_cross_entropy_gradient(y, tx, w, lambda_):
    N = y.shape[0]
    h = sigmoid(tx @ w)
    return 1/N * ((h - y).T @ tx) + lambda_/N * np.sum(w)

In [10]:
def logistic_regression(y, tx, initial_w, max_iters, gamma):
#    batch_size = y.shape[0]
    batch_size = 1
    w = initial_w
    for n_iter in range(max_iters):
        for minibatch_y, minibatch_tx in batch_iter(y, tx, batch_size):
            gr = cross_entropy_gradient(minibatch_y, minibatch_tx, w)
            w = w - gamma * gr
            
        loss = cross_entropy_loss(y, tx, w)
            
#        print("SGD ({bi}/{ti}): loss={l}, w0={w0}, w1={w1}".format(
#              bi=n_iter, ti=max_iters - 1, l=loss, w0=w[0], w1=w[1]))
    return (w, loss)

In [11]:
def reg_logistic_regression(y, tx, lambda_ , initial_w, max_iters, gamma):
#    batch_size = y.shape[0]
    batch_size = 1
    w = initial_w
    for n_iter in range(max_iters):
        for minibatch_y, minibatch_tx in batch_iter(y, tx, batch_size):
            gr = regularized_cross_entropy_gradient(minibatch_y, minibatch_tx, w, lambda_)
            w = w - gamma * gr
            
        loss = regularized_cross_entropy_loss(y, tx, w, lambda_)
            
#        print("SGD({bi}/{ti}): loss={l}, w0={w0}, w1={w1}".format(
#              bi=n_iter, ti=max_iters - 1, l=loss, w0=w[0], w1=w[1]))
    return (w, loss)

In [12]:
# TESTING SCRIPT - IF YOU WANT THAT WEIGHTS ARE EQUAL, YOU SHOULD SET BATCH_SIZE TO Y.SHAPE[0]

# TEST ONLY FOR LOGISTIC REGRESSION WITHOUT REGULARIZATION SINCE SKLEARN HAS DIFFERENT REGULARIZATION LOSS
# FOR REGULARIZED LOGISTIC REGRESSION


from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

X, y = make_classification(n_samples=1000, n_features=4)

In [13]:
LR = LogisticRegression(penalty='none', fit_intercept=False).fit(X, y)
w_official = LR.coef_.flatten()
w_ours, _ = logistic_regression(y, X, np.zeros(X.shape[1]), 50000, 0.6)

print(w_official)
print(w_ours)
print(f"Correct: {np.linalg.norm(w_official - w_ours) < 1e-5}")


[ 1.08722978  2.15555574  0.3857624  -0.98484708]
[ 1.06638775  2.14927062  0.28735909 -0.93775399]
Correct: False
